In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/joaquin/Documents/GitHub/skforecast'

In [2]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset

%load_ext pyinstrument
%load_ext line_profiler

In [3]:
series_1 = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], index=pd.date_range(start='2000-01-01', periods=10, freq='D'), name='series_1')
series_2 = pd.Series([20, 30, 40, 50, 60, 70, 80, 90, 100, 110], index=pd.date_range(start='2000-01-01', periods=10, freq='D'), name='series_2')
series = pd.concat([series_1, series_2], axis=1).melt(var_name='series_id', value_name='value', ignore_index=False)
series = series.rename_axis('datetime')
series = series.reset_index()
series

,datetime,series_id,value
0,2000-01-01,series_1,1
1,2000-01-02,series_1,2
2,2000-01-03,series_1,3
3,2000-01-04,series_1,4
4,2000-01-05,series_1,5
5,2000-01-06,series_1,6
6,2000-01-07,series_1,7
7,2000-01-08,series_1,8
8,2000-01-09,series_1,9
9,2000-01-10,series_1,10


In [4]:
# From long format to long format with multiple and frequancy
# ==================================================================================================
series = series.groupby('series_id').apply(lambda x: x.set_index('datetime').asfreq('D'), include_groups=False)
series

value
series_id datetime         
series_1  2000-01-01      1
          2000-01-02      2
          2000-01-03      3
          2000-01-04      4
          2000-01-05      5
          2000-01-06      6
          2000-01-07      7
          2000-01-08      8
          2000-01-09      9
          2000-01-10     10
series_2  2000-01-01     20
          2000-01-02     30
          2000-01-03     40
          2000-01-04     50
          2000-01-05     60
          2000-01-06     70
          2000-01-07     80
          2000-01-08     90
          2000-01-09    100
          2000-01-10    110

In [5]:
import pandas as pd

def check_frequency_multiseries(series: pd.DataFrame):

    assert isinstance(series.index, pd.MultiIndex)
    assert series.index.nlevels == 2

    series_ids = series.index.get_level_values(0)
    timestamps = series.index.get_level_values(1)
    time_diffs = timestamps.to_series().groupby(series_ids).diff().dropna()
    unique_deltas = np.unique(time_diffs)

    
    if len(unique_deltas) != 1:
        raise ValueError(
            "The time differences within each series are not consistent. "
            "Expected a single unique time difference, but found: "
            f"{time_diffs.drop_duplicates().astype(str).to_numpy()}"
        )


check_frequency_multiseries(series)

In [6]:
indexes_freq = [
    series.loc[series_id].index.freq
    for series_id
    in series.index.get_level_values(0).unique()
]
indexes_freq

[<Day>, <Day>]

In [7]:
series.loc['series_1'].index.freq

<Day>

In [8]:
# Concat series multiindex with exog multiindex equal for all series
# ==================================================================================================
exog = pd.DataFrame({
    'exog_1': [1, 2, 3, 4, 5, 6, 7, 8],
    'exog_2': [10, 20, 30, 40, 50, 60, 70, 80]
}, index=pd.date_range(start='2000-01-01', periods=8, freq='D'))
exog = exog.rename_axis('datetime')
display(exog)

# merge series and exog
series_exog = pd.merge(series, exog, left_index=True, right_index=True, how='left')
series_exog

,exog_1,exog_2
datetime,,
2000-01-01,1,10
2000-01-02,2,20
2000-01-03,3,30
2000-01-04,4,40
2000-01-05,5,50
2000-01-06,6,60
2000-01-07,7,70
2000-01-08,8,80


value  exog_1  exog_2
series_id datetime                         
series_1  2000-01-01      1     1.0    10.0
          2000-01-02      2     2.0    20.0
          2000-01-03      3     3.0    30.0
          2000-01-04      4     4.0    40.0
          2000-01-05      5     5.0    50.0
          2000-01-06      6     6.0    60.0
          2000-01-07      7     7.0    70.0
          2000-01-08      8     8.0    80.0
          2000-01-09      9     NaN     NaN
          2000-01-10     10     NaN     NaN
series_2  2000-01-01     20     1.0    10.0
          2000-01-02     30     2.0    20.0
          2000-01-03     40     3.0    30.0
          2000-01-04     50     4.0    40.0
          2000-01-05     60     5.0    50.0
          2000-01-06     70     6.0    60.0
          2000-01-07     80     7.0    70.0
          2000-01-08     90     8.0    80.0
          2000-01-09    100     NaN     NaN
          2000-01-10    110     NaN     NaN

In [9]:
# Concat series multiindex with exog multiindex diferent exog for each series
# ==================================================================================================
exog_1 = pd.DataFrame({
    'series_id' : 'series_1',
    'exog_1': [1, 2, 3, 4, 5, 6, 7, 8, 9 , 10],
    'exog_2': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}, index=pd.date_range(start='2000-01-01', periods=10, freq='D'))
exog_2 = pd.DataFrame({
    'series_id' : 'series_2',
    'exog_1': [50, 60, 70, 80, 90, 100, 110, 120, 130, 140],
    #'exog_2': [500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400]
}, index=pd.date_range(start='2000-01-01', periods=10, freq='D'))
exog = pd.concat([exog_1, exog_2])
exog = exog.set_index(['series_id', exog.index])
exog.index.names = ['series_id', 'datetime']
exog

# merge series and exog
series_exog = pd.merge(series, exog, left_index=True, right_index=True, how='left')
series_exog

value  exog_1  exog_2
series_id datetime                         
series_1  2000-01-01      1       1    10.0
          2000-01-02      2       2    20.0
          2000-01-03      3       3    30.0
          2000-01-04      4       4    40.0
          2000-01-05      5       5    50.0
          2000-01-06      6       6    60.0
          2000-01-07      7       7    70.0
          2000-01-08      8       8    80.0
          2000-01-09      9       9    90.0
          2000-01-10     10      10   100.0
series_2  2000-01-01     20      50     NaN
          2000-01-02     30      60     NaN
          2000-01-03     40      70     NaN
          2000-01-04     50      80     NaN
          2000-01-05     60      90     NaN
          2000-01-06     70     100     NaN
          2000-01-07     80     110     NaN
          2000-01-08     90     120     NaN
          2000-01-09    100     130     NaN
          2000-01-10    110     140     NaN

In [10]:
forecaster = ForecasterRecursiveMultiSeries(regressor=LGBMRegressor(), lags=5, transformer_exog=StandardScaler())
forecaster.transformer_series_ = {k: StandardScaler() for k in series_exog.index.get_level_values(0).unique()}
forecaster.differentiator_ = {k: None for k in series_exog.index.get_level_values(0).unique()}

series_exog_grouped = series_exog.groupby('series_id', sort=False)
train_matrices = []
for series_id, group in series_exog_grouped:
    group = group.droplevel(0)
    train_matrices.append(
        forecaster._create_train_X_y_single_series(
            y=group.iloc[:, 0].rename(series_id),
            ignore_exog=False,
            exog=group.iloc[:, 1:],
        )
    )

X_train_autoreg = pd.concat([train_matrix[0] for train_matrix in train_matrices])
X_train_window_features_names_out_ = [train_matrix[1] for train_matrix in train_matrices]
X_train_exog = pd.concat([train_matrix[2] for train_matrix in train_matrices])
y_train = pd.concat([train_matrix[3] for train_matrix in train_matrices])

display(X_train_autoreg)
display(X_train_exog)
display(y_train)

,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast
datetime,,,,,,
2000-01-06,-0.174078,-0.522233,-0.870388,-1.218544,-1.566699,series_1
2000-01-07,0.174078,-0.174078,-0.522233,-0.870388,-1.218544,series_1
2000-01-08,0.522233,0.174078,-0.174078,-0.522233,-0.870388,series_1
2000-01-09,0.870388,0.522233,0.174078,-0.174078,-0.522233,series_1
2000-01-10,1.218544,0.870388,0.522233,0.174078,-0.174078,series_1
2000-01-06,-0.174078,-0.522233,-0.870388,-1.218544,-1.566699,series_2
2000-01-07,0.174078,-0.174078,-0.522233,-0.870388,-1.218544,series_2
2000-01-08,0.522233,0.174078,-0.174078,-0.522233,-0.870388,series_2
2000-01-09,0.870388,0.522233,0.174078,-0.174078,-0.522233,series_2


,exog_1,exog_2
datetime,,
2000-01-06,6,60.0
2000-01-07,7,70.0
2000-01-08,8,80.0
2000-01-09,9,90.0
2000-01-10,10,100.0
2000-01-06,100,NaN
2000-01-07,110,NaN
2000-01-08,120,NaN
2000-01-09,130,NaN


datetime
2000-01-06    0.174078
2000-01-07    0.522233
2000-01-08    0.870388
2000-01-09    1.218544
2000-01-10    1.566699
2000-01-06    0.174078
2000-01-07    0.522233
2000-01-08    0.870388
2000-01-09    1.218544
2000-01-10    1.566699
Name: y, dtype: float64

In [11]:
# From long format multiindex to dict
# ==============================================================================
series_ids = series_exog.index.get_level_values(0).unique()
series_dict = {
    sid: series_exog.loc[sid] for sid in series_ids
}
series_dict

{'series_1':             value  exog_1  exog_2
 datetime                         
 2000-01-01      1       1    10.0
 2000-01-02      2       2    20.0
 2000-01-03      3       3    30.0
 2000-01-04      4       4    40.0
 2000-01-05      5       5    50.0
 2000-01-06      6       6    60.0
 2000-01-07      7       7    70.0
 2000-01-08      8       8    80.0
 2000-01-09      9       9    90.0
 2000-01-10     10      10   100.0,
 'series_2':             value  exog_1  exog_2
 datetime                         
 2000-01-01     20      50     NaN
 2000-01-02     30      60     NaN
 2000-01-03     40      70     NaN
 2000-01-04     50      80     NaN
 2000-01-05     60      90     NaN
 2000-01-06     70     100     NaN
 2000-01-07     80     110     NaN
 2000-01-08     90     120     NaN
 2000-01-09    100     130     NaN
 2000-01-10    110     140     NaN}

In [12]:
%%timeit

indexes_dict = {k: g.index.get_level_values(1) for k, g in series.groupby(level=0, sort=False)}
indexes_dict

744 μs ± 34.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
def check_preprocess_series_multi_index(
    series: pd.DataFrame,
) -> tuple[dict[str, pd.Series], dict[str, pd.Index]]:
    """
    Check and preprocess `series` argument in `ForecasterRecursiveMultiSeries` class.

    - If `series` is a pandas DataFrame, it is converted to a dict of pandas 
    Series and index is overwritten according to the rules of preprocess_y.
    - If `series` is a dict, all values are converted to pandas Series. Checks
    if all index are pandas DatetimeIndex and, at least, one Series has a non-null
    frequency. No multiple frequency is allowed.

    Parameters
    ----------
    series : pandas DataFrame, dict
        Training time series.

    Returns
    -------
    series_dict : dict
        Dictionary with the series used during training.
    series_indexes : dict
        Dictionary with the index of each series.
    
    """

    if not isinstance(series.index, pd.MultiIndex):

        _, series_index = preprocess_y(y=series, return_values=False)
        series = series.copy()
        series.index = series_index
        series_dict = series.to_dict("series")

    elif isinstance(series, pd.MultiIndex):
        if not series.index.nlevels == 2:
            raise ValueError(
                f"If `series` is a pandas DataFrame with a MultiIndex, "
                f"it must have two levels (series_id and datetime)."
                f"Found {series.index.names} levels."
            )
        
        if not isinstance(series.index.get_level_values(1), pd.DatetimeIndex):
            raise TypeError(
                f"If `series` is a pandas DataFrame with a MultiIndex, "
                f"the second level (datetime) must be a Pandas DatetimeIndex. "
                f"Found {type(series.index.get_level_values(1))}."
            )


        indexes_freq = [
            series.loc[series_id].index.freq
            for series_id
            in series.index.get_level_values(0).unique()
        ]
        indexes_freq = sorted(set(indexes_freq))
        if not len(indexes_freq) == 1:
            raise ValueError(
                f"If `series` is a dictionary, all series must have a Pandas "
                f"DatetimeIndex as index with the same frequency. "
                f"Found frequencies: {indexes_freq}"
            )
    else:
        raise TypeError(
            f"`series` must be a pandas DataFrame or a dict of DataFrames or Series. "
            f"Got {type(series)}."
        )

    for k, v in series.groupby(level=0, sort=False):
        if v.isna().to_numpy().all():
            raise ValueError(f"All values of series '{k}' are NaN.")

    series_indexes = {k: g.index.get_level_values(1) for k, g in series.groupby(level=0, sort=False)}

    return series_dict, series_indexes